<a href="https://colab.research.google.com/github/opencae/hpc-training-gpu-2023/blob/main/appendix_A_build_rapidcfd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RapidCFDのビルド



## GNU Flexのインストール
RapidCFDのビルドに必要となります。


In [ ]:
!apt-get install flex

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
flex is already the newest version (2.6.4-8build2).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


## インストール先のディレクトリ作成

In [ ]:
!mkdir /root/RapidCFD

## インストール先のディレクトリ移動

In [ ]:
%cd /root/RapidCFD

/root/RapidCFD


## RapidCFD-devのレポジトリのクローン

In [ ]:
!git clone https://github.com/Atizar/RapidCFD-dev.git

Cloning into 'RapidCFD-dev'...
remote: Enumerating objects: 11953, done.
remote: Counting objects: 100% (1004/1004), done.
remote: Compressing objects: 100% (522/522), done.
remote: Total 11953 (delta 484), reused 990 (delta 478), pack-reused 10949
Receiving objects: 100% (11953/11953), 6.76 MiB | 14.14 MiB/s, done.
Resolving deltas: 100% (6435/6435), done.


## パッチファイル入手

In [ ]:
!wget https://gitlab.com/OpenCAE/Supercomputer-OpenFOAM-Training/-/raw/master/build/colab/Nvcc/patches/RapidCFD-dev.diff

--2023-09-08 02:51:15--  https://gitlab.com/OpenCAE/Supercomputer-OpenFOAM-Training/-/raw/master/build/colab/Nvcc/patches/RapidCFD-dev.diff
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3986 (3.9K) [text/plain]
Saving to: ‘RapidCFD-dev.diff’

RapidCFD-dev.diff   100%[===================>]   3.89K  --.-KB/s    in 0s      

2023-09-08 02:51:15 (20.2 MB/s) - ‘RapidCFD-dev.diff’ saved [3986/3986]



## パッチの適用

In [ ]:
!patch -d RapidCFD-dev -b -p1 < RapidCFD-dev.diff

patching file etc/bashrc
patching file src/OpenFOAM/matrices/lduMatrix/preconditioners/AINVPreconditioner/AINVPreconditioner.C
patching file src/OpenFOAM/matrices/lduMatrix/preconditioners/AINVPreconditioner/AINVPreconditioner.H
patching file src/OpenFOAM/meshes/primitiveShapes/objectHit/PointHit.H
patching file src/dynamicFvMesh/solidBodyMotionFvMesh/multiSolidBodyMotionFvMesh.C
patching file wmake/rules/General/mplibSYSTEMOPENMPI


## RapidCFD-devディレクトリ移動

In [ ]:
%cd RapidCFD-dev

/root/RapidCFD/RapidCFD-dev


## コンパイラオプションの変更

ハードウェア・アクセラレータがTesla T4(Turing)の場合には、NVIDIA architecture nameを`sm_75`とします。

* NVIDIA GPU の NVIDIA architectures のまとめ https://qiita.com/k_ikasumipowder/items/1142dadba01b42ac6012

In [ ]:
!for f in wmake/rules/linux64Nvcc/{c,c++};do sed -i 's/ -arch=sm_[0-9]*/ -arch=sm_75/' $f;done

## 環境設定およびビルド
割り当てられたCPUによって、 並列数`WM_NCOMPPROCS`の設定を変えください。
並列数が小さい場合には、非常に時間がかかります。


In [ ]:
!rm platforms/linux64NvccDPOpt/lib/libOpenFOAM.so;\
source etc/bashrc;\
export WM_NCOMPPROCS=2;\
LANG=C ./Allwmake 2>&1 | tee log.Allwmake

make: Nothing to be done for 'all'.
no ThirdParty sources found - skipping
+ wmakePrintBuild -check
same version as previous build
+ wmakeLnInclude OpenFOAM
+ wmakeLnInclude OSspecific/POSIX
+ Pstream/Allwmake
+ wmake libso dummy
'/root/RapidCFD/RapidCFD-dev/platforms/linux64NvccDPOpt/lib/dummy/libPstream.so' is up to date.
+ set +x

Note: ignore spurious warnings about missing mpicxx.h headers

wmake libso mpi
'/root/RapidCFD/RapidCFD-dev/platforms/linux64NvccDPOpt/lib/openmpi-system/libPstream.so' is up to date.
+ OSspecific/POSIX/Allwmake
'/root/RapidCFD/RapidCFD-dev/platforms/linux64NvccDPOpt/lib/libOSspecific.o' is up to date.
+ wmake libso OpenFOAM
'/root/RapidCFD/RapidCFD-dev/platforms/linux64NvccDPOpt/lib/libOpenFOAM.so' is up to date.
+ wmake libso fileFormats
'/root/RapidCFD/RapidCFD-dev/platforms/linux64NvccDPOpt/lib/libfileFormats.so' is up to date.
+ wmake libso surfMesh
'/root/RapidCFD/RapidCFD-dev/platforms/linux64NvccDPOpt/lib/libsurfMesh.so' is up to date.
+ wmake libs

## 単精度版の環境設定およびビルド
もし単精度版をビルドする場合には、以下のようにします。

In [ ]:
!source /root/RapidCFD/RapidCFD-dev/etc/bashrc WM_PRECISION_OPTION=SP;\
export WM_NCOMPPROCS=2;\
LANG=C ./Allwmake 2>&1 | tee log.Allwmake.SP

## RapidCFD-devのアーカイブ


In [ ]:
%cd /root/RapidCFD
!tar Jcf RapidCFD-dev.tar.xz RapidCFD-dev

上記ファイルをファイル・ブラウからダウンロードするか、Googleドライブにコピーしておきます。